In [ ]:
import collections
import os
import pprint

import ee
import pandas as pd

In [ ]:
ee.Initialize(
    project='ee-cmorton',
    opt_url='https://earthengine-highvolume.googleapis.com'
)

In [ ]:
# Remove existing images that are in the skip list
scene_skip_url = 'https://raw.githubusercontent.com/cgmorton/scene-skip-list/main/v2p1_denmark.csv'
scene_skip_list = list(pd.read_csv(scene_skip_url)['SCENE_ID'].values)
scene_skip_list = sorted(scene_skip_list, key=lambda k: k.split('_')[-2], reverse=True)
print(f'Skip list images: {len(scene_skip_list)}')

collections = [
    'projects/openet/assets/ptjpl/eu/era5land/landsat/v2_1',
    'projects/openet/assets/sims/eu/era5land/landsat/v2_1',
    'projects/openet/assets/ssebop/eu/era5land/landsat/v2_1',
]

for coll_id in collections:
    print(f'\n{coll_id}')
    scene_id_list = set(
        ee.ImageCollection(coll_id)
        # .filterDate('2000-01-01', '2015-09-01')
        .aggregate_array('system:index').getInfo()
    )
    print(f'  Images: {len(scene_id_list)}')

    for scene_id in scene_skip_list:
        # print(scene_id)
        if scene_id.lower() in scene_id_list:
            image_id = f'{coll_id}/{scene_id.lower()}'
            print(f'Delete {image_id}')
            try:
                ee.data.deleteAsset(image_id)
            except:
                print('  could not delete asset, skipping')
                continue

print('\nDone')